In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain.agents import tool
from typing import List, Dict, Annotated

In [3]:
from langchain_teddynote.tools import GoogleNews
from langchain_experimental.utilities import PythonREPL

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [5]:
from bs4 import BeautifulSoup
import requests
import re

### 도구 정의

In [6]:
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""

    print(f'검색어: {query}')

    news_tool = GoogleNews()

    return news_tool.search_by_keyword(query, k=2)

In [7]:
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    result = ""
    try:
        result = PythonREPL().run(code)
    except BaseException as e:
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        return result

In [8]:
@tool
def naver_news_crawl(news_url: str) -> str:                 # 네이버 뉴스기사를 크롤링하여 본문 내용을 리턴하는 도구
    """Crawls a 네이버(naver.com) news article and returns the body content."""

    response = requests.get(news_url)                       # HTTP GET 요청

    if response.status_code == 200:                         # 응답 200. 성공
        soup = BeautifulSoup(response.text, 'html.parser')

        title = soup.find('h2', id='title_area').get_text()
        content = soup.find('div', id='contents').get_text()    
        
        cleaned_title = re.sub(r'\n{2, }', '\n', title)
        cleaned_content = re.sub(r'\n{2, }', '\n', content)
    else:
        print(f'HTTP 요청 실패. {response.status_code}')
    
    return f'{cleaned_title}\n{cleaned_content}'

### Agent 프롬프트 작성

In [9]:
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (                                           
            "system",
            "You are a helpful assistant. "             
            "Make sure to use the `search_news` tool for searching keyword related news."
            "Be sure to use the 'python_repl_tool' tool when make python code."
            "Be sure to use the 'naver_news_crawl' tool when naver news crawlling."
        ),
        ("placeholder", "{chat_history}"),          # 이전 대화 내용을 넣을 곳을 잡아둔다.
        ("human", "{input}"),                       # 사용자 입력

        # 에이전트가 검색하는 과정들이나 내용 등을 끄적이는 메모장 같은 공간을 플레이스 홀더로 만들어준다
        ("placeholder", "{agent_scratchpad}"),       
    ]
)

In [10]:
tools = [search_keyword, python_repl_tool, naver_news_crawl]

### Agent 생성

In [11]:
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

agent = create_tool_calling_agent(llm, tools, prompt)

### AgentExecutor

In [12]:
agent_executor = AgentExecutor(
    agent=agent,                    # 각 단계에서 계획을 생성하고 행동을 결정하는 agent
    tools=tools,                    # agent 가 사용할 수 있는 도구 목록
    verbose=True,
    max_iterations=10,              # 최대 10번 까지 반복
    max_execution_time=10,          # 실행되는데 소요되는 최대 시간
    handle_parsing_errors=True      
)

In [13]:
result = agent_executor.stream({'input': '랭체인에 대한 뉴스를 검색해주세요.'})

for step in result:

    if 'actions' in step:
        print('==' * 50)
        print(f'[actions]')
        # print(step['actions'])
        print('[도구 호출]')
        print(f"Tool : {step['actions'][0].tool}")
        print(f"query : {step['actions'][0].tool_input}")
        print(f"Log: ", end='')
        print(f"{step['actions'][0].log}")
        # print(f"message log: {step['actions'][0].message_log}")
        print('==' * 50)

    elif 'steps' in step:
        print('\n')
        print('==' * 50)
        print(f'[steps]')
        # print(step['steps'])
        print(f"검색어: {step['steps'][0].action.tool_input}")
        # print(f"message log: {step['steps'][0].action.message_log}")
        print(f"[관찰 내용]")
        print(f"Observation:")
        print(f"{step['steps'][0].observation}")
        print('==' * 50)

    elif 'output' in step:
        print('==' * 50)
        # print(f"[output]: {step['output']}")
        print(f"[output]")
        print(f"step['messages']:")
        print(f"{step['messages'][0].content}")
        print('==' * 50)



> Entering new None chain...
[actions]
[도구 호출]
Tool : search_keyword
query : {'query': '랭체인'}
Log: 
Invoking: `search_keyword` with `{'query': '랭체인'}`




Invoking: `search_keyword` with `{'query': '랭체인'}`


검색어: 랭체인
[{'url': 'https://news.google.com/rss/articles/CBMibkFVX3lxTE1qMC1EYjRZOEpMNVBEVVpSNm1BX3A4a3FrdmEyOTdDNWdjMGR1ZEgzWlpHZnRFZ1pSUXdKa3JnOXBQZU5qU3dKS3Q5VUNwLVZxRFZmbG1SZzJVSUoyM2lJbnZRbzhYYUJJcnFHWmp3?oc=5', 'content': '랭체인 LangChain 이란 무엇인가? | 인사이트리포트 | 삼성SDS - Samsung SDS'}, {'url': 'https://news.google.com/rss/articles/CBMiwgJBVV95cUxPM0hHcjNGR0RiT0M2QlplRlcxdjdYZE5WTk9zZ2lGVEIxd05hdktEbmRNNTBnLWJ1NEl3LWRPVTNfdFIxX1VOUURxdHdrWTdHOG9LQ3BrOUtWWUl1T0ZianhSckt1NGNtelAweTQ1LXpQTC1DVG50ZUVOOFV1VldBSEZvN2toemJuR3hfQXVfTXBKLVhzWDZMQ25kc1JQM1VnYTRFOTVYSmRFQWJGYWItdTJkNTdXaXlpYzY2dHg0aWM5WVlBQkFRb0N6WGNSWWdVaXhiY1ZXUGJuZ0dpVWtielpJQlV4dFAyZzB6Q2Z0Y1V6SjVZRU5QS0hsRUN0N19EbUc4RXFpV090M2ZZVjZUbS03cC13bFk1M0xrTnJMeHJUOC10dkctbGpJYnFRVHZfVXlQMU5nNDRUbndmbkFiZWFR?oc=5', 'content': '구글

In [14]:
def tool_callback(tool) -> None:
    print(f'===== 도구 호출 =====')
    print(f"사용된 도구 이름 : {tool[0].tool}")
    print(f'===== 도구 호출 =====')

In [15]:
def obervation_callback(observation) -> None:
    print(f'\n===== 관찰 내용 =====')
    print(f"Observation: {observation[0].observation}")
    print(f'===== 관찰 내용 =====')

In [16]:
def result_callback(result: str) -> None:
    print(f'===== 최종 답변 =====')
    print(f"{result}")
    print(f'===== 최종 답변 =====')

In [17]:
result = agent_executor.stream({'input': '랭체인에 대한 뉴스를 검색해주세요'})

for step in result:
    if 'actions' in step:
        tool_callback(step['actions'])
    elif 'steps' in step:
        obervation_callback(step['steps'])
    elif 'output' in step:
        result_callback(step['messages'][0].content)    
        



> Entering new None chain...
===== 도구 호출 =====
사용된 도구 이름 : search_keyword
===== 도구 호출 =====

Invoking: `search_keyword` with `{'query': '랭체인'}`


검색어: 랭체인
[{'url': 'https://news.google.com/rss/articles/CBMibkFVX3lxTE1qMC1EYjRZOEpMNVBEVVpSNm1BX3A4a3FrdmEyOTdDNWdjMGR1ZEgzWlpHZnRFZ1pSUXdKa3JnOXBQZU5qU3dKS3Q5VUNwLVZxRFZmbG1SZzJVSUoyM2lJbnZRbzhYYUJJcnFHWmp3?oc=5', 'content': '랭체인 LangChain 이란 무엇인가? | 인사이트리포트 | 삼성SDS - Samsung SDS'}, {'url': 'https://news.google.com/rss/articles/CBMiwgJBVV95cUxPM0hHcjNGR0RiT0M2QlplRlcxdjdYZE5WTk9zZ2lGVEIxd05hdktEbmRNNTBnLWJ1NEl3LWRPVTNfdFIxX1VOUURxdHdrWTdHOG9LQ3BrOUtWWUl1T0ZianhSckt1NGNtelAweTQ1LXpQTC1DVG50ZUVOOFV1VldBSEZvN2toemJuR3hfQXVfTXBKLVhzWDZMQ25kc1JQM1VnYTRFOTVYSmRFQWJGYWItdTJkNTdXaXlpYzY2dHg0aWM5WVlBQkFRb0N6WGNSWWdVaXhiY1ZXUGJuZ0dpVWtielpJQlV4dFAyZzB6Q2Z0Y1V6SjVZRU5QS0hsRUN0N19EbUc4RXFpV090M2ZZVjZUbS03cC13bFk1M0xrTnJMeHJUOC10dkctbGpJYnFRVHZfVXlQMU5nNDRUbndmbkFiZWFR?oc=5', 'content': '구글, 데이터베이스용 ‘생성형 AI 툴박스’ 공개 베타 발표 "랭체인과 협력" - CIO.com'}]
===== 관

In [18]:
result = agent_executor.stream({'input': '10 + 20을 출력하는 파이썬 코드를 작성하고 실행해주세요'})

for step in result:
    if 'actions' in step:
        tool_callback(step['actions'])
    elif 'steps' in step:
        obervation_callback(step['steps'])
    elif 'output' in step:
        result_callback(step['messages'][0].content)



> Entering new None chain...


Python REPL can execute arbitrary code. Use with caution.


===== 도구 호출 =====
사용된 도구 이름 : python_repl_tool
===== 도구 호출 =====

Invoking: `python_repl_tool` with `{'code': 'print(10 + 20)'}`


30

===== 관찰 내용 =====
Observation: 30

===== 관찰 내용 =====
10 + 20의 결과는 30입니다.

> Finished chain.
===== 최종 답변 =====
10 + 20의 결과는 30입니다.
===== 최종 답변 =====
